# LAB3 - NERC with CRF

### Credits

The content of this notebook is an adaptation of:
https://www.depends-on-the-definition.com/named-entity-recognition-conditional-random-fields-python/

### Background

We denote the input sequence (the words in a sentence):

$$x = (x_1,\dots, x_m)$$

The sequence of output states, i.e. the named entity tags, is represented as:

$$s = (s_1,\dots, s_m)$$

In conditional random fields we model the conditional probability:

$$p(s_1,\dots,s_m|x_1,\dots,x_m)$$

We do this by define a feature map that maps an entire input sequence x paired with an entire state sequence s to some d-dimensional feature vector:

$$\Phi(x_1,\dots,x_m,s_1,\dots,s_m)\in\mathbb{R}^d$$

Then we can model the probability as a log-linear model with the parameter vector `w`:

$$p(s|x; w) = \frac{\exp(w\cdot\Phi(x, s))}{\sum_{s^\prime} \exp(w\cdot\Phi(x, s^\prime))},$$

Here s' ranges over all possible output sequences. For the estimation of w, we assume that we have a set of n labeled examples. Now we define the regularized log-likelihood function L:



$$L(w) = \sum_{i=1}^n \log p(s^i|x^i; w) - \frac{\lambda_2}{2}\|w\|_2^2 - \lambda_1 \|w\|_1.$$

The lambda terms force the parameter vector to be small in the respective norm. This penalizes the model complexity and is known as **regularization**. The parameters lambda_2 and lambda_1 allow us to control the extent of regularization. The parameter vector `w^*` is then estimated as

$$w^* = \text{arg max}_{w\in \mathbb{R}^d} L(w)$$

If we estimated the vector `w^*`, we can find the most likely tag a sentence `s^*` for a sentence x by



$$s^* = \text{arg max}_{s} p(s|x; w^*).$$

### Implementation

#### Step 0: Install the needed modules
1.`sklearn_crfsuite`

Run `pip install sklearn_crfsuite` or 

`conda install -c derickl sklearn-crfsuite`

2.`eli5`

Run `pip install eli5` or 

`conda install -c conda-forge eli5`

#### Step I: Loading the data

Now we want to apply this model. Let’s start by loading the data.

In [2]:
import pandas as pd
import numpy as np

Make sure to download the data from Kaggle first from [this link](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus/downloads/entity-annotated-corpus.zip/4). Note that you will need to register and login in order to do that.

In [3]:
data = pd.read_csv("NERC_datasets/entity-annotated-corpus/ner_dataset.csv", encoding="latin1")

In [ ]:
data = data.fillna(method="ffill")

#### Step II: Initial analysis

Let's print the last 10 rows of the data:

In [ ]:
data.tail(10)

As further analysis, we can make a set of all unique words:

In [ ]:
words = list(set(data["Word"].values))

In [ ]:
n_words = len(words); n_words

So we have 47959 sentences containing 35178 unique words. 

We will use a class called SentenceGetter to retrieve sentences with their labels. Don't worry about the details of this.

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(data)

In [ ]:
sent = getter.get_next()

This is an example sentence we get with our SentenceGetter:

In [ ]:
print(sent)

We can get all sentences as follows:

In [ ]:
sentences = getter.sentences

#### Step III: Feature engineering

Now we craft a set of features and prepare the dataset.

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

The following code extracts features with our functions above. It also prepares all labels from the original dataset.

In [ ]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

#### Step IV: Initialize CRF

Now we can initialize the algorithm. We use the conditional random field (CRF) implementation provided by sklearn-crfsuite.

In [ ]:
import sklearn_crfsuite

from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

5-fold cross-validation.

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report

We will use the scikit-learn classification report to evaluate the tagger, because we are basically interested in precision, recall and the f1-score. These metrics are common in NLP tasks and if you are not familiar with these metrics, then check out the wikipedia articles.

#### Step V: Train and test the CRF algorithm

In [ ]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [ ]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

In [ ]:
crf.fit(X, y)

#### Step VI: Inspect features

The nice thing about CRFs is, that we can look into the algorithm and visualize the transition probabilites from one tag to another. We also can see which features are important for predicting a certain tag. We use the eli5 library to performe the investigation.

In [ ]:
import eli5

In [ ]:
eli5.show_weights(crf, top=30)

#### Step VII: Tuning the model

CRF just remembering a lot of words. For example for the tag ‘B-per’, the algorithm remembers ‘president’ ‘obama’. To overcome this issue we can tune the parameters, especially the regularization parameters of the CRF algorithm. The c1 and c2 parameter of the CRF algorithm are the regularization parameters \lambda_1 and \lambda_2. While c1 weights the l_1 regularization, the c2 parameter weights the l_2 regularization. We now limit the number of features used by enforcing sparsity on the parameter vector w. To do this we increase the l_1-regularization parameter c1.

In [ ]:
crf = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [ ]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

In [ ]:
crf.fit(X, y)

Now we look again at the features.

In [ ]:
eli5.show_weights(crf, top=30)

As expected, we see, that the model stops to rely on words and uses the context more, as it generalizes better is more useful over multiple training instances. This is an effect of the l_1-regularization.

On regularization: "Regularization is a technique to discourage the complexity of the model. It does this by penalizing the loss function. This helps to solve the overfitting problem."

In particular, L1-regularization acts as a feature selector, simply removing some of the features. You can read more on regularization [here](https://medium.com/datadriveninvestor/l1-l2-regularization-7f1b4fe948f2).